In [9]:
import random
import numpy as np
from Bio import Phylo
from Bio.Phylo.Newick import Tree, Clade
from io import StringIO
from collections import defaultdict
from Bio import pairwise2
from Bio.Align import substitution_matrices
import matplotlib.pyplot as plt

# Cargar la matriz BLOSUM62
blosum62 = substitution_matrices.load("BLOSUM62")

# Función de mutación de secuencia (ya definida)
def mutate_sequence(sequence, mutation_rate):
    amino_acids = "ACDEFGHIKLMNPQRSTVWY"
    mutated_sequence = list(sequence)
    
    for i in range(len(sequence)):
        if random.random() < mutation_rate:
            mutated_sequence[i] = random.choice(amino_acids)
    
    return ''.join(mutated_sequence)

# Simular evolución (ya definida)
def simulate_evolution(sequence, num_leaf_sequences, mutation_rate, extinction_prob=0.1, speciation_prob=0.2):
    sequences = {0: sequence}  
    tree_history = [(0, None)] 
    next_id = 1
    alive_sequences = {0}  
    mutation_counts = defaultdict(int)
    
    while len(alive_sequences) < num_leaf_sequences:
        to_mutate = list(alive_sequences)
        
        for species in to_mutate:
            # aqui se aplica la mutacion
            new_sequence = mutate_sequence(sequences[species], mutation_rate)
            mutation_counts[(species, next_id)] = sum(1 for a, b in zip(sequences[species], new_sequence) if a != b)
            
            # aqui es donde hay la speciacion
            if random.random() < speciation_prob:
                sequences[next_id] = new_sequence
                alive_sequences.add(next_id)
                tree_history.append((next_id, species))
                next_id += 1
            
            # extincion
            if random.random() < extinction_prob and len(alive_sequences) > 1:
                alive_sequences.remove(species)
    
    return sequences, tree_history, mutation_counts

# Generar árbol Newick (ya definido)
def generate_newick(tree_history):
    clades = {i: Clade(name=str(i)) for i, _ in tree_history}
    root = None
    
    for child, parent in tree_history:
        if parent is not None:
            clades[parent].clades.append(clades[child])
        else:
            root = clades[child]
    
    tree = Tree(root)
    return tree

# Exportar árbol en formato Newick (ya definido)
def export_tree_to_newick(tree):
    output = StringIO()
    Phylo.write(tree, output, 'newick')
    return output.getvalue()

# Calcular la distancia BLOSUM62 entre secuencias
def blosum62_distance(seq1, seq2):
    alignments = pairwise2.align.globalds(seq1, seq2, blosum62, -10, -0.5)
    score = alignments[0][2] 
    return -score  

# Dibujar árbol usando matplotlib
def draw_tree(tree):
    Phylo.draw(tree)

# Graficar comparaciones de distancia
def plot_distances(original_distances, new_distances):
    plt.figure(figsize=(10, 6))
    plt.hist(original_distances, bins=20, alpha=0.5, label='Original')
    plt.hist(new_distances, bins=20, alpha=0.5, label='Rearranged')
    plt.legend(loc='upper right')
    plt.title('Comparison of Distances Before and After Tree Rearrangement')
    plt.xlabel('Distance')
    plt.ylabel('Frequency')
    plt.show()

# Función principal para la evaluación
# Aquí, asumimos que se utiliza un método de reorganización codicioso (o se puede agregar un algoritmo genético según tus necesidades)
# La función de reorganización es actualmente un marcador de posición (puedes implementarla por separado)

def evaluate_tree_rearrangement(tree, distance_matrix):
    original_distance = np.sum(distance_matrix)  # Suma de todas las distancias por pares 
    new_distance = original_distance - 10  # Simulando que la reorganización mejora la distancia en un valor (marcador de posición)
    
    print(f"Original distance: {original_distance}")
    print(f"Rearranged distance: {new_distance}")
    
    return original_distance, new_distance

# Implementar el algoritmo de parsimonia
def parsimony_score(tree, sequences):
    score = 0
    for clade in tree.find_clades(order='postorder'):
        if clade.is_terminal():
            clade.state = set(sequences[int(clade.name)])
        else:
            child_states = [child.state for child in clade.clades]
            intersection = set.intersection(*child_states)
            if intersection:
                clade.state = intersection
            else:
                clade.state = set.union(*child_states)
                score += 1
    return score

# Generar árboles aleatorios y calcular longitudes de árboles verdaderas y estimadas
def generate_random_trees(num_trees, num_leaf_nodes):
    trees = []
    for _ in range(num_trees):
        tree_history = [(i + 1, random.randint(0, i)) for i in range(num_leaf_nodes - 1)]
        tree_history.insert(0, (0, None))  
        trees.append(generate_newick(tree_history))
    return trees

def calculate_tree_lengths(trees, sequences):
    true_lengths = []
    estimated_lengths = []
    
    for tree in trees:
        true_length = sum(len(sequences[int(clade.name)]) for clade in tree.get_terminals())
        estimated_length = parsimony_score(tree, sequences)
        
        true_lengths.append(true_length)
        estimated_lengths.append(estimated_length)
    
    return true_lengths, estimated_lengths

def main():
    num_trees = 50
    num_leaf_nodes = 20
    
    evolutionary_distances = [0.5, 1.0, 2.0]
    
    for distance in evolutionary_distances:
        start_sequence = "ACDEFGHIKLMNPQRSTVWY" * int(100 * distance)
        sequences, _, _ = simulate_evolution(start_sequence, num_leaf_nodes, mutation_rate=0.05)
        
        trees = generate_random_trees(num_trees, num_leaf_nodes)
        
        true_lengths, estimated_lengths = calculate_tree_lengths(trees, sequences)
        
        differences = np.array(true_lengths) - np.array(estimated_lengths)
        
        mean_difference = np.mean(differences)
        std_difference = np.std(differences)
        
        print(f"Evolutionary Distance: {distance * 100}%")
        print(f"Mean Difference: {mean_difference}")
        print(f"Standard Deviation of Difference: {std_difference}")
        print()

if __name__ == "__main__":
    main()


Evolutionary Distance: 50.0%
Mean Difference: 10120.0
Standard Deviation of Difference: 1423.2357499725756

Evolutionary Distance: 100.0%
Mean Difference: 19960.0
Standard Deviation of Difference: 2979.664410634191

Evolutionary Distance: 200.0%
Mean Difference: 41120.0
Standard Deviation of Difference: 5935.1158371172505

